# **Build Recommendation System with Content-Based Filtering**

# Packages

Import Packages



In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf

# Import Dataset


In [11]:
user_dataset = pd.read_csv("./user_data.csv")
event_dataset= pd.read_csv("./event_data.csv")

In [28]:
print(len(user_dataset))
print(len(event_dataset))

4987
1092


# Preprocessing Data User


In [ ]:
# Preprocessing user data
user_dataset['Skills'] = user_dataset['Skills'].str.lower()
user_dataset['Availability'] = user_dataset['Availability'].str.lower()
user_dataset['Location'] = user_dataset['Location'].str.lower()
user_dataset['Type of Organization'] = user_dataset['Type of Organization'].str.lower()

# Memisahkan menjadi beberapa kolom terpisah untuk one hot
user_skills_split = user_dataset['Skills'].str.split(', ', expand=True)

# Create one-hot encoding for user skills
user_skills_one_hot = pd.get_dummies(user_skills_split, prefix='Skill')

user_skills_one_hot

In [ ]:
# Merge Onehot encoding with data user
user_data_encode = pd.concat([user_dataset, user_skills_one_hot], axis=1)
user_data_encode

# Preprocessing Data Event


In [29]:
## Preprocessing event data
#event_dataset['Kualifikasi'] = event_dataset['Kualifikasi'].str.lower()

## Create one-hot encoding for event category
# event_category_one_hot = pd.get_dummies(event_dataset['Kategori'], prefix='Category')

In [ ]:
# Merge user and event data
merged_data = pd.merge(user_dataset, event_dataset, how='cross')

# Create one-hot encoding for user and event data
user_one_hot = pd.get_dummies(merged_data[['Age', 'Gender', 'Availability', 'Location', 'Type of Organization']], prefix='User')
event_one_hot = pd.get_dummies(merged_data[['Age', 'Gender', 'Domisili']], prefix='Event')

# Concatenate user and event one-hot encodings
X = pd.concat([user_skills_one_hot, user_one_hot, event_category_one_hot, event_one_hot], axis=1)

# Target variable
y = merged_data['Match']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Nyoba proses nilai umur



```
###Pemrosesan Data Umur
def process_age(value):
    if '-' in str(value):  # Jika nilai adalah rentang umur
        age_range = value.split('-')
        return (int(age_range[0]) + int(age_range[1])) / 2
    elif isinstance(value, int):  # Jika nilai adalah umur tunggal dan sudah integer
        return value
    else:
        # Penanganan lainnya
        return None


event_dataset['Age'] = event_dataset['Age'].apply(process_age)

```

